In [1]:
### Run this notebook using cell -> run all
### Demo of one solution to 'blocking' until a widget value has changed

In [11]:
import time
import sys

class CaptureExecution:
    "A context manager to capture cell execution events then replay them after exiting the manager"
    def __init__(self):
        self.captured_events = []
        self.shell = get_ipython()
        self.kernel = self.shell.kernel
        
    def step(self):
        "Call the kernel loop sort of manually to read messages (cell execution/widget updates)"
        self.kernel.do_one_iteration() # same thing an eventloop like %gui asyncio would do
        #time.sleep(.01)
    
    def capture_event(self, stream, ident, parent):
        self.captured_events.append((stream, ident, parent))

    def start_capturing(self):
        "Overwrite the kernel shell handlers"
        self.kernel.shell_handlers['execute_request'] = self.capture_event

    def stop_capturing(self):
        "rever the kernel shell handler functions to their defaults"
        self.kernel.shell_handlers['execute_request'] = self.kernel.execute_request
    
    def replay_captured_events(self):
        sys.stdout.flush() # without flush here then output goes into the wrong cell
        for stream, ident, parent in self.captured_events:
            self.kernel.set_parent(ident, parent)
            self.kernel.execute_request(stream, ident, parent)
        self.captured_events = []
    
    def __enter__(self):
        self.start_capturing()
        self.shell.execution_count += 1 # increment execution count to avoid database collision error
        
    def __exit__(self, *args):
        self.stop_capturing()
        self.replay_captured_events()

        
def capture_requests(widget=None, break_func=None):
    """
    A function to capture execution requests.  
    It manages the CaptureExecution context manager, just tell it when to break
      param: widget -- context manager will exit whenever the widget value changes
      param: break_func -- context manager will exit when the break_func returns True
    """
    if widget is None and break_func is None:
        raise Exception("need to pass in a widget to watch to change or a break function to run")
    ctx = CaptureExecution()
    with ctx:
        while True:
            if widget is not None:
                if widget.value:
                    break
            else:
                if break_func():
                    break
            ctx.step()

In [15]:
import ipywidgets as widgets

w = widgets.Dropdown(options=['', 'foo', 'bar'])
    
display(w)

capture_requests(w)

Dropdown(options=('', 'foo', 'bar'), value='')

In [13]:
print(w.value)

foo


In [14]:
# this should get captured in a cell -> run all 
print("hello world")

hello world


In [7]:
w2 = widgets.Dropdown(options=['', 'foo', 'bar'])
display(w2)


def f():
    return w2.value == 'bar'

capture_requests(break_func=f)


Dropdown(options=('', 'foo', 'bar'), value='')

In [8]:
print(w2.value)

bar


In [10]:
%%while

UsageError: Cell magic `%%while` not found.
